In [1]:
import torch
import cv2

from S3FD.data.config import cfg
from S3FD.s3fd_model import build_s3fd
from S3FD.utils.augmentations import to_chw_bgr

from common.det_face import DetFace

- 모델 로드

In [2]:
def load_model(phase='test', device='cpu'):
    net = build_s3fd(phase, cfg.NUM_CLASSES)

    net.load_state_dict(torch.load('./S3FD/weights/sfd_face.pth', map_location=torch.device(device)))
    net.eval()
    return net

mode = 'test'
model = load_model(phase=mode, device='cpu')

- 테스트

In [3]:
def detect_faces(__model, frame, thresh=0.2):
    img_orig = frame
    img = cv2.cvtColor(img_orig.copy(), cv2.COLOR_BGR2RGB)

    image = img
    x = to_chw_bgr(image)
    x = x.astype('float32')
    x -= cfg.img_mean
    x = x[[2, 1, 0], :, :]

    x = torch.from_numpy(x).unsqueeze(0)

    with torch.no_grad():
        y = __model(x)
    detections = y.data

    img = img_orig.copy()
    scale = torch.Tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])

    det_faces = []
    for i in range(detections.size(1)):
        j = 0
        while detections[0, i, j, 0] >= thresh:
            pt = (detections[0, i, j, 1:] * scale).cpu().numpy()
            score = detections[0, i, j, 0].cpu().numpy()

            det_faces.append(DetFace(float(score), (pt[0], pt[1], pt[2], pt[3])))
            j += 1

    return det_faces

In [4]:
def display_result(image_path, model):
    image_path = "./images/test_image.jpg"
    frame = cv2.imread(image_path)
    faces = detect_faces(model, frame)

    # 시각화
    for face in faces:
        x_min, y_min, x_max, y_max = map(int, face.bbox)
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)  # 경계 상자
        cv2.putText(frame, f"{face.conf:.2f}", (x_min, y_min - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)  # 신뢰도
    
    # 결과 이미지 보기
    cv2.imshow('Detected Faces', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
image_path = "./images/test_image.jpg"
frame = cv2.imread(image_path)
faces = detect_faces(model, frame)

# 얼굴 감지 결과 출력
for face in faces:
    print(f"Confidence: {face.conf}, Bounding Box: {face.bbox}")

Confidence: 0.9999998807907104, Bounding Box: (424.65997, 124.141426, 688.69727, 469.66187)
Confidence: 0.3810512125492096, Bounding Box: (653.3873, 755.29443, 915.21954, 1012.1224)
Confidence: 0.29773664474487305, Bounding Box: (822.1539, 805.07446, 1321.1066, 1316.6844)
Confidence: 0.23724570870399475, Bounding Box: (810.61646, -41.50844, 1287.7639, 458.92633)
Confidence: 0.21913231909275055, Bounding Box: (933.8792, 928.1095, 1173.6028, 1170.0208)


C:\Users\eunhak\workspace\face_detection\S3FD\layers\bbox_utils.py:287: UserWarning: An output with one or more elements was resized since it had shape [97], which does not match the required output shape [76]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\Resize.cpp:35.)
  torch.index_select(x1, 0, idx, out=xx1)
C:\Users\eunhak\workspace\face_detection\S3FD\layers\bbox_utils.py:288: UserWarning: An output with one or more elements was resized since it had shape [97], which does not match the required output shape [76]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, 

In [6]:
# 시각화
for face in faces:
    x_min, y_min, x_max, y_max = map(int, face.bbox)
    cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)  # 경계 상자
    cv2.putText(frame, f"{face.conf:.2f}", (x_min, y_min - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)  # 신뢰도

# 결과 이미지 보기
cv2.imshow('Detected Faces', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

- FGSM

In [7]:
mode = 'attack'
model = load_model(phase=mode, device='cpu')

In [9]:
image_path = "./images/test_image.jpg"
frame = cv2.imread(image_path)
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
image_tensor = torch.from_numpy(frame).permute(2, 0, 1).unsqueeze(0).float() / 255.0  # Normalize to [0, 1]

output = model(image_tensor)
decoded_boxes, conf_preds = output

print(decoded_boxes.shape, conf_preds.shape)

torch.Size([1, 96803, 4]) torch.Size([1, 2, 96803])


In [28]:
import torch
import torch.nn.functional as F
import numpy as np

def fgsm_attack(image, epsilon, gradient):
    # 그래디언트의 부호를 기반으로 입력 데이터 교란
    sign_gradient = gradient.sign()
    perturbed_image = image + epsilon * sign_gradient
    
    # 이미지를 [0, 1] 범위로 클리핑
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

def attack_s3fd(model, image, epsilon):
    """
    FGSM 공격을 통해 S3FD 모델의 신뢰도를 교란.
    Args:
        model: S3FD 모델
        image: 입력 이미지 (Tensor, shape: [1, 3, H, W])
        epsilon: 노이즈 강도
    Returns:
        perturbed_image: 공격 후 이미지
    """
    # 입력 데이터에 그래디언트 활성화
    image.requires_grad = True

    # 모델 예측 수행
    decoded_boxes, conf_preds = model(image)

    # loss = -torch.max(conf_preds).mean() #########

    target_class = 1  # 예: 클래스 1
    loss = -conf_preds[:, target_class, :].mean() ######## 잘 모르겠음
    
    loss.backward()

    # FGSM 공격 수행
    gradient = image.grad.data
    perturbed_image = fgsm_attack(image, epsilon, gradient)

    return perturbed_image



In [29]:
# 원본 이미지 로드
image_path = "./images/test_image.jpg"
frame = cv2.imread(image_path)
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

resized_frame = cv2.resize(frame, (300, 300))

image_tensor = torch.from_numpy(resized_frame).permute(2, 0, 1).unsqueeze(0).float() / 255.0  # Normalize to [0, 1]

model.eval()

# FGSM 공격
epsilon = 0.05  # 노이즈 강도
perturbed_image = attack_s3fd(model, image_tensor, epsilon)

# 공격 후 결과 확인
perturbed_frame = perturbed_image.squeeze().permute(1, 2, 0).detach().numpy()
perturbed_frame = (perturbed_frame * 255).astype('uint8')

# RGB → BGR 변환
perturbed_frame = cv2.cvtColor(perturbed_frame, cv2.COLOR_RGB2BGR)
# 원본 이미지도 BGR 형식으로 변환
original_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

In [30]:
# 결과 비교
cv2.imshow("Original Image", original_frame)
cv2.imshow("Perturbed Image", perturbed_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()